# Convolutional Neural Network for CIFAR 10

In [1]:
import tensorflow as tf
import numpy as np
import math
import os
import cPickle

In [2]:
import math

# Creating the Model

#### Description

In [3]:
dataset_root = os.path.abspath("../dataset/cifar-10-batches-py/")
dataset_root

'/home/fjcamillo/Desktop/cnn_cifar10/dataset/cifar-10-batches-py'

In [4]:
dataset = [dataset_root+"/"+data for data in os.listdir(dataset_root) if "data_batch" in data]
dataset

['/home/fjcamillo/Desktop/cnn_cifar10/dataset/cifar-10-batches-py/data_batch_3',
 '/home/fjcamillo/Desktop/cnn_cifar10/dataset/cifar-10-batches-py/data_batch_4',
 '/home/fjcamillo/Desktop/cnn_cifar10/dataset/cifar-10-batches-py/data_batch_5',
 '/home/fjcamillo/Desktop/cnn_cifar10/dataset/cifar-10-batches-py/data_batch_1',
 '/home/fjcamillo/Desktop/cnn_cifar10/dataset/cifar-10-batches-py/data_batch_2']

In [5]:
with open(dataset[3], "r") as fo:
    train = cPickle.load(fo)
    
with open(dataset[4], "r") as fo:
    test = cPickle.load(fo)

In [6]:
train["data"].shape

(10000, 3072)

In [7]:
np.array(train["labels"]).shape

(10000,)

In [8]:
training_data = [dat.reshape((32,32,3)) for dat in train["data"]]

In [9]:
training_label = np.array(train["labels"])

In [10]:
test_data = [dat.reshape((32,32,3)) for dat in test["data"]]

In [11]:
test_label = np.array(test["labels"])

In [12]:
test_label[:200]

array([1, 6, 6, 8, 8, 3, 4, 6, 0, 6, 0, 3, 6, 6, 5, 4, 8, 3, 2, 6, 0, 3, 1,
       4, 0, 6, 6, 2, 7, 6, 9, 0, 4, 5, 7, 1, 6, 7, 9, 1, 7, 7, 8, 0, 3, 7,
       4, 7, 3, 1, 0, 4, 6, 6, 1, 4, 9, 2, 6, 4, 5, 0, 4, 6, 0, 8, 3, 4, 8,
       8, 3, 9, 5, 7, 1, 9, 4, 7, 9, 1, 9, 7, 5, 2, 7, 3, 4, 8, 8, 2, 1, 5,
       9, 2, 7, 8, 8, 6, 8, 8, 1, 3, 8, 8, 5, 4, 7, 1, 6, 6, 1, 6, 1, 6, 7,
       0, 4, 6, 9, 5, 8, 7, 1, 9, 0, 3, 3, 7, 6, 9, 0, 4, 7, 1, 4, 3, 4, 3,
       9, 8, 7, 0, 8, 3, 9, 1, 0, 8, 0, 9, 4, 0, 2, 1, 4, 2, 7, 1, 7, 0, 2,
       9, 7, 9, 8, 6, 4, 4, 1, 1, 7, 8, 8, 6, 4, 5, 6, 3, 9, 4, 2, 5, 3, 6,
       7, 7, 3, 9, 1, 3, 1, 8, 3, 3, 5, 6, 4, 0, 9, 6])

In [37]:
#Helper Function to convert the data to one hot encode]
def convert_onehot(data):
    converted = []
    for i in xrange(10):
        if data == i:
            converted.append(1)
        else: converted.append(0)
    return converted

In [38]:
final_test = np.array([convert_onehot(dat) for dat in list(test_label)], dtype=np.float32)

In [39]:
final_test[31]

array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.], dtype=float32)

In [40]:
final_training = np.array([convert_onehot(dat) for dat in list(training_label)], dtype=np.float32)

# Model

In [13]:
size = (32,32,3)

In [14]:
strides = [1, 1, 1, 1]
stddev = 1.0/math.sqrt(32*32)

In [15]:
with tf.device("/cpu:0"):
    
    x = tf.placeholder(tf.float32, [None, 32, 32, 3])
    y_ = tf.placeholder(tf.float32, [None])
    W1 = tf.Variable(tf.truncated_normal([6,6,3,8],stddev=stddev))
    
    B1 = tf.Variable(tf.zeros(8))
    W2 = tf.Variable(tf.truncated_normal([5,5,8,12],stddev=stddev))
    B2 = tf.Variable(tf.zeros(12))
    W3 = tf.Variable(tf.truncated_normal([4,4,12,14],stddev=stddev))
    B3 = tf.Variable(tf.zeros(14))
    W4 = tf.Variable(tf.truncated_normal([32*32*14,300],stddev=stddev))
    B4 = tf.Variable(tf.zeros(300))
    W5 = tf.Variable(tf.truncated_normal([300,10],stddev=stddev))
    B5 = tf.Variable(tf.zeros(10))
    
    #First Convolutional Neural Network
    conv1 = tf.nn.relu(tf.nn.conv2d(x, W1, strides=strides, padding="SAME")+B1)
    
    #Second Convolutional Neural Network
    conv2 = tf.nn.relu(tf.nn.conv2d(conv1,W2, strides=strides, padding="SAME")+B2)
    
    #Third Convolutional Neural Network
    conv3 = tf.nn.relu(tf.nn.conv2d(conv2,W3, strides=strides, padding="SAME")+B3)
    
    #Reshape and flatten the convolutional layers into a fully connected layer
    conv3_reshaped = tf.reshape(conv3,[-1,32*32*14])
    
    #First Fully Connected Layer
    logits4 = tf.add(tf.matmul(conv3_reshaped, W4), B4)
    fc1 = tf.nn.relu(logits4)
    
    #Second Fully Connected Layer
    logits5 = tf.add(tf.matmul(fc1, W5),B5)
    fc2 = tf.nn.softmax_cross_entropy_with_logits(logits=logits5, labels=y_)
    loss = tf.reduce_mean(fc2)

    train = tf.train.AdamOptimizer(0.001).minimize(loss)
    
    #Accuracy
    correct_prediction = tf.equal(tf.argmax(y_,1),tf.argmax(fc2, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [17]:
sess = tf.InteractiveSession()

In [18]:
sess.run(tf.global_variables_initializer())

In [35]:
for i in range(200):
    batch = 200
    if i%100==0:
        train_accuracy = accuracy.eval(feed_dict={x:training_data[:batch],y_:final_training[:batch]})
        print("Step {}: \tTraining Accuracy = {}".format(i, train_accuracy))
    train.run(feed_dict={x:training_data[:batch],y_:final_training[:batch]})

print("Final Accuracy: {}".format(accuracy.eval(feed_dict={x:test_data[:batch],y_:final_test[:batch]})))

ValueError: Cannot feed value of shape (200, 9) for Tensor u'Placeholder_1:0', which has shape '(?,)'